# Backend

In [1]:
import nltk
import numpy as np
import regex as re
import csv
import json
import pickle
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### 1. Preparación


In [3]:


stemmer = SnowballStemmer('english')
# abrir el csv
list = [] # Guarda las canciones
nombres = [] # Nombres de las canciones
i = 0
with open(r'..\Postgres\spotify_songs.csv', newline='', encoding='utf-8') as file:
    songs_file = csv.reader(file, delimiter=',', quotechar='"')
    for row in songs_file:
        # track_name , artist_name, lyrics
        song_details = row[1] + " " + row[2] + " " + row[3]  
        nombres.append(row[1])
        list.append(song_details)
 
  

list.pop(0)
nombres.pop(0)

print(list[0])
print(nombres[0])

Pangarap Barbie's Cradle Minsan pa Nang ako'y napalingon Hindi ko alam Na ika'y tutugon Sa mga tanong na aking nabitawan Hindi ko alam kung ito'y totoo Pangarap ka Sa bawat sandali Langit man ang tingin ko Sayo sana'y marating Hanggang dito na lang yata Ang kaya kong gawin Mangarap na lang At bumulong sa hangin Kailan kaya Darating ulit ang isang Sandali Na ako'y lilingon muli Pangarap ka o tinig mong kay lamig Ang iyong mga ngiti na sa akin ay Nakapagbigay pansin (Ikaw ba ay isang pangarap lang) Pangarap ka o tinig mong kay lamig Ang iyong mga ngiti Na sa akin ay Nakapagbigay... Pangarap ka o tinig mong kay lamig Ang iyong mga ngiti Na sa akin ay Nakapagbigay Pangarap ka o tinig mong kay lamig Ang iyong mga ngiti Na sa akin ay Nakapagbigay pansin
Pangarap


In [ ]:
for i in nombres:
    print(i)

### 1- Preprocesamiento

In [17]:
# Tokenizar, filtrar stopwords, eliminar signos, stemming

with open("stoplist.txt", encoding="latin-1") as file:
    stoplist = [line.rstrip().lower() for line in file]
    
def preprocesamiento(texto,stemming=True):
  words = []
  # 1- convertir a minusculas
  texto = texto.lower()
  # 2- eliminar signos con regex
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # 3- tokenizar
  words = nltk.word_tokenize(texto, language='english')
  # 3- eliminar stopwords
  words = [word for word in words if word not in stoplist]
  # 4- Aplicar reduccion de palabras (stemming)
  if stemming:        
      words = [stemmer.stem(word) for word in words]
  return words

In [18]:
# Preprocesar las canciones
textos_procesados = [] # lista de listas de palabras de cada cancion
# indice = {}

for i in range(0, len(list)):
  texto = preprocesamiento(list[i])
  textos_procesados.append(texto)


In [ ]:
for i in range(0, len(textos_procesados)):
  print(textos_procesados[i])

### 2- Construcción del índice TF-IDF

In [19]:
def compute_tfidf(collection):
    # Calcular los pesos TF-IDF para cada documento de la colección
    N = len(collection)
    
    # Listar todos los términos en la coleccion
    terms = set()
    for doc in collection:
        for word in doc:
            terms.add(word)
    # print(1)

    # Calcular la frecuencia de palabras en cada documento usando el set de todas las palabras (terms)
    # tf
    freqs = [] 
    for doc in collection:
        freq = {}
        for word in doc:
            if word in terms:
                freq[word] = freq.get(word, 0) + 1
        freqs.append(freq)
    # print(2)

    # Calcular el IDF para cada término
    idfs = {word: 0 for word in terms}
    for doc in collection:
        unique_words_in_doc = set(doc)
        for word in unique_words_in_doc:
            idfs[word] += 1
    # print(3)

    # Calcular IDF para todos los términos
    for word in idfs:
        idfs[word] = np.log(N / idfs[word])
    # print(4)

    # Calcular los pesos TF-IDF para cada documento
    tfidf = []
    for i, doc in enumerate(collection):
        doc_tfidf = {}
        for word in freqs[i]:  # Solo calcular TF-IDF para términos presentes en el documento
            tf = freqs[i][word] # cantidad de frecuancia por palabra
            doc_tfidf[word] = tf * idfs[word]
        tfidf.append(doc_tfidf)

    # print(5)
    return tfidf, freqs


In [20]:
textos_tfidf, tf_list = compute_tfidf(textos_procesados)


In [ ]:
# imprimir los tf
for i in tf_list:
    print(i)

In [ ]:
# imprimir los pesos de los términos
# for i in range(0, len(textos_tfidf)):
#   print(textos_tfidf[i])

Guardarlo en memoria secundaria

#### Construcción del índice en memoria secundaria (SPIMI)

In [ ]:
# diccionario1 = {"1":["doc1"], "3":["doc1", "doc3"]}
# diccionario2 = {"2":["doc1", "doc2", "doc3"]}

# block_size = 3
# output = {"1":["doc1"], "2":["doc1", "doc2"]}
# output = {"2":["doc3"], "3":["doc1", "doc3"]}

In [21]:
filename = "data.dat"
textos_procesados_size = len(textos_procesados)

dictionary_num = round(np.log2(textos_procesados_size)) # numero de diccionario generado
print(dictionary_num)
docs_num = np.ceil(textos_procesados_size/dictionary_num)
print(docs_num)



bucket_count = 0 # cantidad de buckets

block_size = 1 * 2**20 # MB
# print(block_size)

14
1319.0


In [22]:
class Bucket:
    def __init__(self, lista = {}, next=None):
        self.lista = lista # {doc: tf} # 5 bytes
        self.next = next  # Siguiente bloque (memoria) # 4 bytes

    def Write(self):
        with open(filename, 'ab') as f:
            p = f.tell()
            pickle.dump(self, f)
            return p

    @staticmethod
    def Read(bucket):
        with open(filename, 'rb') as f:
            f.seek(bucket)
            return pickle.load(f)



    def add_doc(self, doc, tf): # añadir un doc y tf
        dictionary = {doc: tf}
        self.lista[doc] = dictionary

    def insert_next_bucket(self):
        self.next = Bucket()
    
    def next_bucket(self, next): # siguiente bucket
        self.next = next
        return self.next

    def bucket_size(self): # tamaño del bucket
        serializado = pickle.dumps(self.lista) + pickle.dumps(self.next)
        return len(serializado)

    def get_list(self): # get list
        return self.lista

    def sort_bucket(self):
        self.lista = dict(sorted(self.lista.items(), key=lambda item: item[1], reverse=True))


In [23]:
class Dictionary:
    def __init__(self, terminos={}): # {word: [df, bucket]}
        self.terminos = terminos

    def Write(self):
        # with open(filename, 'ab') as f:
        #     self.bucket = f.tell()    # Marca el lugar de escritura en el archivo
        #     pickle.dump(self, f)
        bucket = Bucket()
        for termino in self.terminos:
            # ingresar a los buckets
            bucket = self.terminos[termino][1]
            
            while (bucket != None):
                bucket.Write()
                bucket = bucket.next

    @staticmethod
    def Read(bucket):
        with open(filename, 'rb') as f:
            f.seek(bucket)
            return pickle.load(f)



    def insert_term(self, termino): # inserta un termino
        if termino not in self.terminos: # si no esta en el diccionario
            self.terminos[termino] = [1, Bucket()] # añadir un bucket
        else: # si esta en el diccionario
            self.incrementar_df(termino) # incrementa el df
            


    def insert_doc(self, termino, doc, tf, bucket):
        if bucket.bucket_size() < block_size: # si el tamaño del bucket es menor al limite
            bucket.add_doc(doc, tf) # añadir documento en el bucket
        else: # ir al siguiente bucket
            if (bucket.next == None): # si no hay siguiente bucket
                bucket.insert_next_bucket() # crear siguiente bucket
            else: # si hay siguiente bucket
                self.insert_doc(self, termino, doc, tf, self.terminos[termino][1].next)



    def incrementar_df(self, termino):
        self.terminos[termino][0] += 1
        
    # def insert_bucket(self, termino, doc, tf):
    #     self.terminos[termino][1] = Bucket()

    def sort_dictionary(self): # ordenar los terminos
        self.terminos = dict(sorted(self.terminos.items(), key=lambda item: item[0]))
    

In [24]:
def mergeBlocks(fn):
    for i in range(0, len(fn)):
        # Merge blocks
        # leer dos bloques de memoria secundaria
        input1 = fn[i].Read(filename, i)
        input2 = fn[j].Read(filename, j)
        output = {}
        
        # merge
        for j in range(0, int(np.log2(n))): # cantidad de niveles
            for k in range(0, int(n/2)): # cantidad de bloques
                output[input1.keys()[k]] = input1.values()[k] + input2.values()[k]
        # 

    return output

In [25]:
def SPIMI_Invert(docs, doc_num): # terminos de un documento
    dictionary = Dictionary() # Dictionary / (word, df, bucket) / diccionario local / guarda el enlace a la cabecera de lista de bloques de publicaciones (posting_list)
    # posting_list = Bucket()  # Bucket / lista de publicaciones / lista de {doci, tf}
    pos = 0
    
    # while (len(terminos)>pos): # mientras hay terminos, se llena el diccionario
    for doc in docs:
        for termino in doc: # terminos del doc i
            pos+=1

            dictionary.insert_term(termino) # añadir un termino al diccionario

            dictionary.insert_doc(termino, nombres[doc_num], tf_list[doc_num][termino], dictionary.terminos[termino][1]) # añadir el documento
            
            # posting_list = dictionary.terminos[termino][1] # obtener el Bucket del termino

            # if (dictionary.terminos[termino][1].bucket_size() > block_size): # si el tamaño del bloque es mayor al tamaño del bloque
            #     dictionary.next_bucket(termino) # añadir un bucket
            
            # docID = nombres[doc_num]
            
        doc_num += 1

    # Ordenar el diccionario
    dictionary.sort_dictionary()

    # Escribir los buckets en memoria secundaria / el diccionario en RAM
    dictionary.Write()

    return dictionary

BSBIndex

In [ ]:
# print(len(textos_procesados))
# print(round(np.log2(len(textos_procesados))))

18454
14


In [26]:
fn = [] # lista de diccionarios locales / indices invertidos locales (key, df, bucket)


def BSBIndex():
    docs = []
    i = 0
    doc_num = 0 # que documento se está procesando

    for doc in textos_procesados: # juntar canciones
        docs.append(doc)
        i+=1

        if i >= docs_num or i == textos_procesados_size: 
            fn.append(SPIMI_Invert(docs, doc_num))
            docs = []
        

    mergeBlocks(fn) # todos los diccionarios/indices locales

In [27]:
BSBIndex()

KeyError: 'i'

# Búsqueda

In [ ]:
def cosine_sim(Q, Doc):
    # Obtener los términos comunes entre los dos documentos
    terms_comun = set(Q.keys()).intersection(set(Doc.keys()))
    
    # Calcular el producto punto
    dot_product = sum(Q[term] * Doc[term] for term in terms_comun)
    
    # Calcular las normas (longitudes) de los vectores
    norm_Q = np.sqrt(sum(value**2 for value in Q.values()))
    norm_Doc = np.sqrt(sum(value**2 for value in Doc.values()))
    
    # Evitar división por cero
    if norm_Q == 0 or norm_Doc == 0:
        return 0.0
    
    # Retornar la similitud de coseno
    return dot_product / (norm_Q * norm_Doc)

In [ ]:
#Similitud de coseno en memoria secundaria
# q: texto de consulta
# min_sim: similitud minima
def cosineScore(q, min_sim=0.8, filename, N):
    float Scores[N]=0
    float Length[N]
    float result = []
    for i in range(0, N): # itera por cada una de las paginas
        posting_list = read(filename, i) # toma una pagina i del disco
        for t in q: # itera los terminos de la query y lo compara con los de la posting_list
            calculate_wtq(posting_list, t)
            for d, tf in posting_list:
                Scores[d] += wtd*wtq

    Length = size(posting_list) # asigna las longitudes de cada documento
    for d in posting_list: # itera por los documentos de la posting_list
        Scores[d] = Scores[d]/Length[d]
        if Scores[d] > min_sim: # inserta los documentos que cumplen con ser mayores al min_sim
            result.append(d)
